In [47]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from skorch import NeuralNetClassifier
from torch.optim.lr_scheduler import ReduceLROnPlateau, CyclicLR
from skorch.callbacks import LRScheduler
from sklearn.metrics import accuracy_score
from scipy.fft import fft
from sklearn.ensemble import RandomForestClassifier

In [19]:
def signal0(samples_per_signal, noise_amplitude):
    x = np.linspace(0, 4.0, samples_per_signal)
    y = np.sin(x * np.pi * 0.5)
    n = np.random.randn(samples_per_signal) * noise_amplitude
    
    s = y + n
    
    shift = np.random.randint(low=0, high=int(samples_per_signal / 2))
    s = np.concatenate([s[shift:], s[:shift]])
    
    return np.asarray(s, dtype=np.float32)
    

def signal1(samples_per_signal, noise_amplitude):
    x = np.linspace(0, 4.0, samples_per_signal)
    y = np.sin(x * np.pi)
    n = np.random.randn(samples_per_signal) * noise_amplitude
    
    s = y + n
    
    shift = np.random.randint(low=0, high=int(samples_per_signal / 2))
    s = np.concatenate([s[shift:], s[:shift]])
    
    return np.asarray(s, dtype=np.float32)



class Network(nn.Module):
    
    def __init__(self, signal_size):
        
        c = int(signal_size / 10)
        if c < 3:
            c = 3
        
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 8, c),
            nn.ReLU(),
            #nn.Dropout1d(0.1),
            nn.AvgPool1d(2),
            nn.Conv1d(8, 16, c),
            nn.ReLU(),
            #nn.Dropout1d(0.1),
            nn.AvgPool1d(2),
            #nn.Conv1d(16, 1, 1),
            #nn.Dropout1d(0.1),
            nn.ReLU(),
            nn.Flatten()
        )
        
        l = 0
        with torch.no_grad():
            s = torch.randn((1,1,SAMPLES_PER_SIGNAL))
            o = self.cnn(s)
            l = o.shape[1]
        
        self.head = nn.Sequential(
            nn.Linear(l, 2 * l),
            nn.ReLU(),
            #nn.Dropout1d(0.1),
            nn.Linear(2 * l, 2),
            nn.ReLU(),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        
        x = self.cnn(x)
        x = self.head(x)
        
        return x    

In [51]:
SAMPLES_PER_SIGNAL = 100
SIGNALS_IN_DATASET = 20
NOISE_AMPLITUDE = 0.1
REPEAT_EXPERIMENT = 10

X, Y = [], []

stop = int(SIGNALS_IN_DATASET / 2)
for i in range(SIGNALS_IN_DATASET):

    if i < stop:
        x = signal0(SAMPLES_PER_SIGNAL, NOISE_AMPLITUDE)
        y = 0
    else:
        x = signal1(SAMPLES_PER_SIGNAL, NOISE_AMPLITUDE)
        y = 1

    X.append(x.reshape(1,-1))
    Y.append(y)

X = np.concatenate(X)
Y = np.array(Y, dtype=np.int64)

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.1)

In [24]:
accs = []
train_accs = []

for i in range(REPEAT_EXPERIMENT):

    net = NeuralNetClassifier(
        lambda: Network(SAMPLES_PER_SIGNAL),
        max_epochs=200,
        criterion=nn.CrossEntropyLoss(),
        lr=0.1,
        callbacks=[
            #('lr_scheduler', LRScheduler(policy=ReduceLROnPlateau, monitor="valid_acc", mode="min", verbose=True)),
            ('lr_scheduler', LRScheduler(policy=CyclicLR, base_lr=0.0001, max_lr=0.01, step_size_up=10)),
        ],
        verbose=False,
        batch_size=128
    )

    net = net.fit(train_x.reshape(train_x.shape[0], 1, SAMPLES_PER_SIGNAL), train_y)
    pred = net.predict(test_x.reshape(test_x.shape[0], 1, SAMPLES_PER_SIGNAL))
    acc = accuracy_score(test_y, pred)
    
    print(f"{i} - {acc}")


    accs.append(acc)
    
    pred_train = net.predict(train_x.reshape(train_x.shape[0], 1, SAMPLES_PER_SIGNAL))
    train_acc = accuracy_score(train_y, pred_train)
    train_accs.append(train_acc)
    
    print(f"Train Acc: {train_acc}, Test Acc: {acc}")
    
accs = np.array(accs)
train_accs = np.array(train_accs)

print(f"Average acc: {accs.mean()}")
print(f"Average train acc: {train_accs.mean()}")
print(f"Average acc where training was successful: {accs[train_accs > 0.6].mean()}")
print(f"Training success rate: {(train_accs > 0.6).mean()}")

0 - 0.55
Train Acc: 0.49444444444444446, Test Acc: 0.55
1 - 1.0
Train Acc: 1.0, Test Acc: 1.0
2 - 1.0
Train Acc: 1.0, Test Acc: 1.0
3 - 0.55
Train Acc: 0.49444444444444446, Test Acc: 0.55
4 - 1.0
Train Acc: 1.0, Test Acc: 1.0
5 - 0.55
Train Acc: 0.49444444444444446, Test Acc: 0.55
6 - 1.0
Train Acc: 1.0, Test Acc: 1.0
7 - 1.0
Train Acc: 1.0, Test Acc: 1.0
8 - 0.95
Train Acc: 0.9722222222222222, Test Acc: 0.95
9 - 1.0
Train Acc: 1.0, Test Acc: 1.0
Average acc: 0.8600000000000001
Average train acc: 0.8455555555555556
Average acc where training was successful: 0.9928571428571429
Training success rate: 0.7


In [52]:
X, Y = [], []

stop = int(SIGNALS_IN_DATASET / 2)
for i in range(SIGNALS_IN_DATASET):

    if i < stop:
        x = signal0(SAMPLES_PER_SIGNAL, NOISE_AMPLITUDE)
        y = 0
    else:
        x = signal1(SAMPLES_PER_SIGNAL, NOISE_AMPLITUDE)
        y = 1

    x = np.abs(fft(x[:int(SAMPLES_PER_SIGNAL /2 )]))    
    
    X.append(x.reshape(1,-1))
    Y.append(y)

X = np.concatenate(X)
Y = np.array(Y, dtype=np.int64)

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.1)

accs = []
train_accs = []

for i in range(REPEAT_EXPERIMENT):
    model = RandomForestClassifier()
    model.fit(train_x, train_y)
    
    pred = model.predict(test_x)
    acc = accuracy_score(test_y, pred)
    
    print(f"{i} - {acc}")


    accs.append(acc)
    
    pred_train = model.predict(train_x)
    train_acc = accuracy_score(train_y, pred_train)
    train_accs.append(train_acc)
    
    print(f"Train Acc: {train_acc}, Test Acc: {acc}")
    
accs = np.array(accs)
train_accs = np.array(train_accs)

print(f"Average acc: {accs.mean()}")
print(f"Average train acc: {train_accs.mean()}")
print(f"Average acc where training was successful: {accs[train_accs > 0.6].mean()}")
print(f"Training success rate: {(train_accs > 0.6).mean()}")

0 - 1.0
Train Acc: 1.0, Test Acc: 1.0
1 - 1.0
Train Acc: 1.0, Test Acc: 1.0
2 - 1.0
Train Acc: 1.0, Test Acc: 1.0
3 - 1.0
Train Acc: 1.0, Test Acc: 1.0
4 - 1.0
Train Acc: 1.0, Test Acc: 1.0
5 - 1.0
Train Acc: 1.0, Test Acc: 1.0
6 - 1.0
Train Acc: 1.0, Test Acc: 1.0
7 - 1.0
Train Acc: 1.0, Test Acc: 1.0
8 - 1.0
Train Acc: 1.0, Test Acc: 1.0
9 - 1.0
Train Acc: 1.0, Test Acc: 1.0
Average acc: 1.0
Average train acc: 1.0
Average acc where training was successful: 1.0
Training success rate: 1.0
